In [56]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [57]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [58]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

In [59]:
indice = parametros.mintic_concat_index
# indice

In [60]:
# indice='dev-mintic-concat'

### Función para generar JSON compatible con ES

In [61]:
def filterKeys(document,key):
    return {key: document[key] for key in use_these_keys }

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [62]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
           "_source": ["gestion.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_gestion_incidentes"
                    }
                  }
              ]
            }
          }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['gestion.fechaControl']
except Exception as e:
    print(e)
    response["hits"]["hits"] = []
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01T00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-01-01T16:30:00


### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede.)

In [63]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob','energiadesc'
                           ,'nombreSede','latitud', 'longitud','id_Beneficiario','COD_ISO','codDanesede',
                           'cod_servicio','codDaneMuni','nombre_centro_pob','codCentroPoblado','codDaneInstitucionEdu',
                           'tipoSitio','detalleSitio','energia','region','matricula','DDA','grupoDesc','estadoInstalacion']
            
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    
    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except Exception as e:
    print(e)
    print("fecha:",now,"- Error en lectura de datos semilla")
    exit()

In [64]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))

datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['gestion.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['gestion.location']=datos_semilla['gestion.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

#datos_semilla

### Leyendo indice servicemanager-incidentes

Se lee la información de los De Service Manager. En la lectura se traen todas las interacciones reportadas por servicemanager, con la información de la misma.

* clr_bmc_location, es la llave para cruzar con cada centro de conexión.
* product_type, es el detalle del ticket.
* clr_bmc_host, estado la IP
* assignment, es el sujeto asignado al ticket
* contact_name, usuario del ticket
* open_time y resolved_time, son las fecha de apertura y cierre del ticket
* resolution, respuesta al ticket
* number, numero del ticket
* source, fuente del ticket(email o telefono)
* severity, gravedad del ticket
* category y subcategory, clasificacion dada por mintic

* Se consulta SM incidentes, que tengan fecha de resolucion dentro del rango procesado
* la duración de la falla es la diferencia en horas de resolved_time y open_time, menos los tiempos de parada
* el id del incidente en el indice incidentes es number, en interacciones es incident_id

In [65]:
def traeSMIncidentes(fecha_max,fecha_tope):
    total_docs = 10000
    try:
        response = es.search(
            index= parametros.sm_incidentes,
            body={
                "_source": ["clr_bmc_location","product_type","clr_bmc_host","resolution","contact_name"
                            ,"assignment","open_time","resolved_time","resolution","number","severity"]
                ,"query": {
                    "range": {
                      "@timestamp": {
                        "gte": fecha_max,
                        "lt": fecha_tope
                      }
                    }
                }
            },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]
#         fields = {}
#         for num, doc in enumerate(elastic_docs):
#             source_data = doc["_source"]
#             for key, val in source_data.items():
#                 try:
#                     fields[key] = np.append(fields[key], val)
#                 except KeyError:
#                     fields[key] = np.array([val])
                    

#         datos_SM_incidents = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
        
        datos_SM_incidents = pd.DataFrame([x["_source"] for x in elastic_docs])
        
        
        datos_SM_incidents['gestion.categoria'] = 'Dano' 
        datos_SM_incidents['gestion.subcategoria'] = 'Dano masivo internet' 
        datos_SM_incidents = datos_SM_incidents.rename(columns={'severity':'gestion.gravedad',
                                                                'product_type' : 'gestion.detallesTicket',
                                                                'contact_name':'gestion.usuarioTicket',|
                                                                'assignment':'gestion.responsable',
                                                                'clr_bmc_location':'site_id',
                                                                'clr_bmc_host':'gestion.IP',
                                                                'number':'gestion.numeroTicket'})
        #datos_SM_incidents['gestion.tiempoRespuesta'] = datos_SM_incidents.apply(lambda row: row.resolved_time - row.open_time, axis=1)
        if not datos_SM_incidents.empty:
            datos_SM_incidents['gestion.gravedad'] = datos_SM_incidents['gestion.gravedad'].replace(['1','2','3'],['Alto','Medio','Bajo'])
#             datos_SM_incidents['gestion.duracion'] = datos_SM_incidents.apply(lambda row: row.resolved_time - row.open_time if( row.resolved_time is not None) else 0, axis=1)
        '''
        ### Por si no sirve ese calculo de horas
        aaa['f1'] = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S") for x in concat["gestion.fechaApertura"]]
        aaa['f2'] = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S") for x in concat["gestion.fechaControl"]]
        aaa['f3'] = ((aaa['f1']-aaa['f2']).dt.seconds)/3600
        '''
        return datos_SM_incidents
    except Exception as e:
        print(e)
        return pd.DataFrame()
    

SyntaxError: invalid syntax (<ipython-input-65-59e9ace12637>, line 40)

In [66]:
def calTotalParada(total):
    #total="4000-01-01T05:01:51.000Z"
    #Retorna el total parada en horas
    try:
        ts = datetime.strptime(total,"%Y-%m-%d"'T'"%H:%M:%S.%fZ")
        return ((ts.month-1)*30*24*60+(ts.day-1)*24*60+(ts.hour)*60+(ts.minute))/60
    except:
        return 0

def calHoras(delta):
    #Retorna la cantidad de horas
    try:
        resp = delta.days*24+(delta.seconds)/(3600)
    except:
        resp = ''
    return resp

def convFecha(fecha,formato):
    try:
        resp = datetime.strptime(fecha,formato)
    except:
        resp = np.nan
    return resp

def duracionEnDias(x):
    try:
        resp = math.trunc(float(x)/24)
    except:
        resp = np.nan
    return resp

def duracionEnHoras(x):
    try:
        resp = math.trunc((float(x)/24-math.trunc(float(x)/24))*24)
    except:
        resp = np.nan
    return resp

In [67]:
# (t1-t0).days*24+((t1-t0).seconds)/(3600)

# calTotalParada("4000-01-01T05:01:51.000Z")

# 3600/(24*60)

# 2021-06-23T17:53:25.000Z	None	2021-07-03T06:05:47.000Z
#2021-06-01T09:27:34.000Z	None	2021-06-15T17:06:12.000Z	4000-01-01T05:24:07.000Z

In [68]:
open_time=datetime.strptime("2021-06-17 17:09:48","%Y-%m-%d %H:%M:%S")

In [69]:
resolved_time=datetime.strptime("2021-06-17 17:33:14","%Y-%m-%d %H:%M:%S")

In [70]:
diferencia=resolved_time - open_time
diferencia

datetime.timedelta(seconds=1406)

In [71]:
1406/60

23.433333333333334

In [72]:
diferencia

datetime.timedelta(seconds=1406)

In [73]:
import numpy as np
calHoras(np.NaN)

''

In [74]:
ch=calHoras(diferencia)
float(ch)

0.39055555555555554

In [75]:
total= "4000-01-13T04:54:31.000Z"
tp=calTotalParada(total)
tp

292.9

In [76]:
ch-tp

-292.5094444444444

In [77]:
# parametros.sm_paradas = 'servicemanager-reloj'

In [78]:
dias=125/24
dias

5.208333333333333

In [79]:
horas = 125%24
horas

5

In [80]:
minutos=(125/24-125%24)*24
minutos

4.999999999999993

In [81]:
import math
math.trunc(0.2083333*24)

4

In [82]:
125/24

5.208333333333333

In [83]:
dias=125

In [84]:
math.trunc(dias/24)

5

In [85]:
math.trunc((dias/24-math.trunc(dias/24))*24)

4

In [86]:
math.trunc(((dias/24-math.trunc(dias/24))*24-math.trunc((dias/24-math.trunc(dias/24))*24))*60)

59

In [87]:
def traeSMParadas(fecha_max,fecha_tope,filtro_nombres = ["PENDIENTE CLIENTE",
                                                         "SUSPENDIDO",
                                                         "PENDIENTE PROVEEDOR",
                                                         "PENDING CUSTOMER", 
                                                         "SUSPENDED", 
                                                         "PENDING SUPPLIER"]):
    total_docs = 10000
    
    try:
        response = es.search(
            index= parametros.sm_paradas,
            body={
                  "_source":["open_time","resolved_time","total","clr_txt_idbneficmtc","key_char","name"],
                  "query":{
                    "range":{
                      "open_time":{
                        "gte": fecha_max ,
                        "lt": fecha_tope
                      }
                    }
                  }

                },
            size=total_docs
        )
        elastic_docs = response["hits"]["hits"]

        datos_SM_paradas = pd.DataFrame([x["_source"] for x in elastic_docs])
        
        datos_SM_paradas = datos_SM_paradas.rename(columns={'open_time':'gestion.incidente.fechaApertura',
                                                            'resolved_time' : 'gestion.incidente.fechaResolucion',
                                                            'total':'gestion.incidente.totalParadas',
                                                            'clr_txt_idbneficmtc':'gestion.incidente.id_Beneficiario',
                                                            'key_char':'gestion.incidente.numeroTicket',
                                                            'incident_id':'gestion.incidente.numeroTicket'})
        
        #Filtrado por campo "name" para los valores:
        #
        #   - Pendiente cliente
        #   - Pendiente proveedor
        #   - Suspendido
        
        
        filtro_nombres = [x.upper() for x in filtro_nombres]
        
        if filtro_nombres != ['*']:
            datos_SM_paradas=datos_SM_paradas[datos_SM_paradas["name"].str.upper().isin(filtro_nombres)]
                                           
#         #Eliminar todos los registros sin fecha de apertura del caso..
#         datos_SM_paradas.drop(datos_SM_paradas[pd.isna(datos_SM_paradas['gestion.incidente.fechaApertura'])].index,inplace=True)
#         #Eliminar todos los registros sin fecha de resolucion del caso..
#         datos_SM_paradas.drop(datos_SM_paradas[pd.isna(datos_SM_paradas['gestion.incidente.fechaResolucion'])].index,inplace=True)
#         #Eliminar todos los registros sin el total de paradas del caso..
#         datos_SM_paradas.drop(datos_SM_paradas[pd.isna(datos_SM_paradas['gestion.incidente.totalParadas'])].index,inplace=True)
        
        #datos_SM_incidents['gestion.tiempoRespuesta'] = datos_SM_incidents.apply(lambda row: row.resolved_time - row.open_time, axis=1)
        if not datos_SM_paradas.empty:
            formato = "%Y-%m-%d"'T'"%H:%M:%S.%fZ"
            datos_SM_paradas['gestion.incidente.totalParadas2'] = datos_SM_paradas['gestion.incidente.totalParadas']
            datos_SM_paradas['gestion.incidente.totalParadas'] = datos_SM_paradas['gestion.incidente.totalParadas'].\
                                                                 apply(lambda fecha:calTotalParada(fecha))
            
            datos_SM_paradas['gestion.incidente.duracion'] =  datos_SM_paradas['gestion.incidente.fechaResolucion'].\
                                                              apply(lambda fecha: convFecha(fecha,formato))- \
                                                              datos_SM_paradas['gestion.incidente.fechaApertura'].\
                                                              apply(lambda fecha: convFecha(fecha,formato))
            #Verificar dtype de datos_SM_paradas['gestion.incidente.duracion']
            
#             datos_SM_paradas['gestion.incidente.duracion2'] =  datos_SM_paradas['gestion.incidente.duracion']
            
            datos_SM_paradas['gestion.incidente.duracionTotalHrs'] =  datos_SM_paradas['gestion.incidente.duracion'].\
                                                                      apply(lambda delta: calHoras(delta))-\
                                                                      datos_SM_paradas['gestion.incidente.totalParadas']
#             datos_SM_paradas['gestion.incidente.duracionTotalHrs'].astype('float')
            datos_SM_paradas['gestion.incidente.duracionDias'] = datos_SM_paradas['gestion.incidente.duracionTotalHrs'].\
                                                                 apply(lambda x: duracionEnDias(x))
            datos_SM_paradas['gestion.incidente.duracionHoras'] = datos_SM_paradas['gestion.incidente.duracionTotalHrs'].\
                                                                  apply(lambda x: duracionEnHoras(x))
#             datos_SM_paradas['gestion.incidente.duracionMinutos'] = datos_SM_paradas['gestion.incidente.duracion'].\
#                                                                   apply(lambda x:math.trunc(((x/24-math.trunc(x/24))*24-\
#                                                                                  math.trunc((x/24-math.trunc(x/24))*24))*60))
            #Se toma la minima duracion que incluye la totalidad de las paradas
            datos_SM_paradas = datos_SM_paradas.loc[datos_SM_paradas.groupby(['gestion.incidente.fechaApertura',\
                                                                              'gestion.incidente.fechaResolucion'])\
                                                                             ['gestion.incidente.duracion'].idxmin()]
            
            
        '''
        ### Por si no sirve ese calculo de horas
        aaa['f1'] = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S") for x in concat["gestion.fechaApertura"]]
        aaa['f2'] = [datetime.strptime(x, "%Y-%m-%d %H:%M:%S") for x in concat["gestion.fechaControl"]]
        aaa['f3'] = ((aaa['f1']-aaa['f2']).dt.seconds)/3600
        '''
        return datos_SM_paradas
    except Exception as e:
        print(e)
        return pd.DataFrame(columns=['gestion.incidente.fechaApertura',
                                     'gestion.incidente.fechaResolucion',
                                     'gestion.incidente.totalParadas',
                                     'gestion.incidente.id_Beneficiario',
                                     'gestion.incidente.numeroTickett'
                                    ])

Realizando bucle hasta conseguir datos de servicemanager-incidentes o hasta la fecha actual para realizar la carga de datos

In [88]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)

In [89]:
fecha_max_mintic,fecha_tope_mintic

('2021-01-01T16:30:00', '2021-01-01T16:39:59')

In [90]:
fecha_max_mintic="2021-04-01T00:00:00"
fecha_tope_mintic="2021-07-24T00:00:00"
datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)

In [91]:
datos_SM_incidents

,gestion.gravedad,gestion.numeroTicket,gestion.usuarioTicket,gestion.detallesTicket,gestion.responsable,gestion.IP,open_time,site_id,resolved_time,resolution,gestion.categoria,gestion.subcategoria
0,Alto,IM1278675,bmcintegrationsm,hardware failure,EYN - NOCMINTIC,172.28.107.185,2021-04-23T19:15:15.000Z,31699-ZGYO293,None,None,Dano,Dano masivo internet
1,Alto,IM1278675,bmcintegrationsm,hardware failure,EYN - NOCMINTIC,172.28.107.185,2021-04-23T19:15:15.000Z,31699-ZGYO293,None,None,Dano,Dano masivo internet
2,Alto,IM1278675,bmcintegrationsm,hardware failure,EYN - NOCMINTIC,172.28.107.185,2021-04-23T19:15:15.000Z,31699-ZGYO293,None,None,Dano,Dano masivo internet
3,Alto,IM1278675,bmcintegrationsm,hardware failure,EYN - NOCMINTIC,172.28.107.185,2021-04-23T19:15:15.000Z,31699-ZGYO293,None,None,Dano,Dano masivo internet
4,Alto,IM1278675,bmcintegrationsm,hardware failure,EYN - NOCMINTIC,172.28.107.185,2021-04-23T19:15:15.000Z,31699-ZGYO293,None,None,Dano,Dano masivo internet
5,Alto,IM1278675,bmcintegrationsm,hardware failure,EYN - NOCMINTIC,172.28.107.185,2021-04-23T19:15:15.000Z,31699-ZGYO293,None,None,Dano,Dano masivo internet


In [92]:
if datos_SM_incidents is None or datos_SM_incidents.empty:
    while (datos_SM_incidents is None or datos_SM_incidents.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_incidents = traeSMIncidentes(fecha_max_mintic,fecha_tope_mintic)
else:
    pass
#datos_SM_incidents

Realizando bucle hasta conseguir datos de <b>servicemanager-reloj</b> o hasta la fecha actual para realizar la carga de datos

In [93]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)-timedelta(seconds=1)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
datos_SM_paradas = traeSMParadas(fecha_max_mintic,fecha_tope_mintic)

module 'parametros' has no attribute 'sm_paradas'


In [94]:
datos_SM_paradas

,gestion.incidente.fechaApertura,gestion.incidente.fechaResolucion,gestion.incidente.totalParadas,gestion.incidente.id_Beneficiario,gestion.incidente.numeroTickett


In [95]:
if datos_SM_paradas is None or datos_SM_paradas.empty:
    while (datos_SM_paradas is None or datos_SM_paradas.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d"'T'"%H:%M:%S")) < str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")
        datos_SM_paradas = traeSMParadas(fecha_max_mintic,fecha_tope_mintic)
else:
    pass
#datos_SM_incidents

module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


...

module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'
module 'parametros' has no attribute 'sm_paradas'


In [96]:
#No puede tomarse una ventana de 10 min porque puede que en ese intervalo no esta 
#la resolucion del caso o no esten incluidas todas las paradas

In [97]:
fecha_max_mintic, fecha_tope_mintic

('2021-08-16T00:00:00', '2021-08-16T00:09:59')

In [98]:
fecha_max_mintic='2021-01-01T16:20:00'
fecha_tope_mintic='2021-07-25T00:00:00'

In [99]:
datos_SM_paradas = traeSMParadas(fecha_max_mintic,fecha_tope_mintic,['*'])
datos_SM_paradas

module 'parametros' has no attribute 'sm_paradas'


,gestion.incidente.fechaApertura,gestion.incidente.fechaResolucion,gestion.incidente.totalParadas,gestion.incidente.id_Beneficiario,gestion.incidente.numeroTickett


In [100]:
datos_SM_paradas.shape

(0, 5)

In [101]:
datos_SM_paradas['name'].unique()

KeyError: 'name'

In [ ]:
datos_SM_paradas[datos_SM_paradas['gestion.incidente.numeroTicket']=='SD1399968'][['gestion.incidente.fechaApertura','gestion.incidente.fechaResolucion', 'gestion.incidente.totalParadas2','gestion.incidente.duracionDias', 'gestion.incidente.duracionHoras']]

In [ ]:
datos_SM_paradas.shape

In [ ]:
datos_SM_paradas[pd.isna(datos_SM_paradas['gestion.incidente.id_Beneficiario'])][['gestion.incidente.numeroTicket','gestion.incidente.id_Beneficiario']]

In [ ]:
datos_SM_paradas[datos_SM_paradas['gestion.incidente.duracionTotalHrs']>0][['gestion.incidente.numeroTicket','gestion.incidente.id_Beneficiario','gestion.incidente.fechaApertura','gestion.incidente.fechaResolucion','gestion.incidente.totalParadas2','gestion.incidente.duracionDias', 'gestion.incidente.duracionHoras']]

In [ ]:
datos_SM_paradas[(datos_SM_paradas['gestion.incidente.duracionTotalHrs']<0) & (datos_SM_paradas['gestion.incidente.numeroTicket']=='SD1414522')]

Haciendo merge entre semilla e incidentes

In [ ]:
try:
    concat = pd.merge(datos_SM_incidents,datos_semilla, on=['site_id'],how='inner')
    concat = concat.rename(columns={'id_Beneficiario' : 'gestion.id_Beneficiario'
                                    ,'resolution':'gestion.resolucion'
                                    ,'variable3':'gestion.Tipo3'
                                    ,'variable4':'gestion.Tipo4'
                                    ,'nombreInstitucionEd':'gestion.nombreInstitucionEd'
                                    ,'open':'gestion.abierto'
                                    ,'open_time':'gestion.fechaApertura'
#                                     ,'resolved_time':'gestion.fechaCierre'
                                    ,'nombreSede':'gestion.nombreSede'
                                    ,'site_id':'gestion.site_id'
                                    ,'nombre_departamento':'gestion.dptoGestion'
                                    ,'nombre_municipio':'gestion.muniGestion'
                                    ,'nombre_centro_pob':'gestion.nombre_centro_pob'
                                    ,'COD_ISO':'gestion.COD_ISO'
                                    ,'codDanesede':'gestion.codDanesede'
                                    ,'energiadesc':'gestion.energiadesc'
                                    ,'cod_servicio':'gestion.cod_servicio'
                                    ,'codDaneMuni':'gestion.codDaneMuni'
                                    ,'nombre_centro_pob':'gestion.nombre_centro_pob'
                                    ,'codCentroPoblado':'gestion.codCentroPoblado'
                                    ,'codDaneInstitucionEdu':'gestion.codDaneInstitucionEdu'
                                    ,'tipoSitio':'gestion.tipoSitio'
                                    ,'detalleSitio':'gestion.detalleSitio'
                                    ,'energia':'gestion.energia'
                                    ,'region':'gestion.region'
                                    ,'matricula':'gestion.matricula'
                                    ,'DDA':'gestion.DDA'
                                    ,'grupoDesc':'gestion.grupoDesc'
                                    ,'estadoInstalacion':'gestion.estadoInstalacion'                                                                   
                                   })
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.replace("T"," ")
    concat['gestion.fechaApertura'] = concat['gestion.fechaApertura'].str.slice(stop=19)
    
    concat['gestion.fechaControl'] = concat["gestion.fechaApertura"].str[0:-4] + '0:00'
    
#     concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.replace("T"," ")
#     concat['gestion.fechaCierre'] = concat['gestion.fechaCierre'].str.slice(stop=19)
    concat["gestion.anyo"] = concat["gestion.fechaApertura"].str[0:4]
    concat["gestion.mes"] = concat["gestion.fechaApertura"].str[5:7]
    concat["gestion.dia"] = concat["gestion.fechaApertura"].str[8:10]
    concat["gestion.hora"] = concat["gestion.fechaApertura"].str[11:13]
    concat["gestion.min"] = concat["gestion.fechaApertura"].str[14:16]
    concat["gestion.seg"] = concat["gestion.fechaApertura"].str[17:19]
    
    concat["gestion.estadoTicket"]= "Cerrado" if isinstance(datetime.strptime(concat['resolved_time'], "%Y-%m-%d"'T'"%H:%M:%S")) else "Abierto"
    
except Exception as e:
    print(e)
    concat = pd.DataFrame()

In [ ]:
concat

Realizando inserción

In [ ]:
use_these_keys = ['gestion.numeroTicket'
                  , 'gestion.usuarioTicket'
                  , 'gestion.estadoTicket'
                  , 'gestion.categoria'
                  , 'gestion.subcategoria'
                  , 'gestion.fechaApertura'
                  , 'gestion.fechaCierre'
                  , 'gestion.responsable'
                  , 'gestion.location'
                  , 'gestion.canal'
                  , 'gestion.id_Beneficiario'
                  , 'gestion.resolucion'
                  , 'gestion.Tipo3'
                  , 'gestion.Tipo4'
                  , 'gestion.nombreInstitucionEd'
                  , 'gestion.abierto'
                  , 'gestion.nombreSede'
                  , 'gestion.COD_ISO'
                  , 'gestion.site_id'
                  , 'gestion.dptoGestion'
                  , 'gestion.muniGestion'
                  , 'gestion.energiadesc'
                  , 'gestion.gravedad'
                  , 'gestion.anyo'
                  , 'gestion.mes'
                  , 'gestion.dia'
                  , 'gestion.hora'
                  , 'gestion.min'
                  , 'gestion.seg'
                  , 'gestion.cod_servicio'
                  , 'gestion.codDaneMuni'
                  , 'gestion.nombre_centro_pob'
                  , 'gestion.codCentroPoblado'
                  , 'gestion.codDaneInstitucionEdu'
                  , 'gestion.tipoSitio'
                  , 'gestion.detalleSitio'
                  , 'gestion.energia'
                  , 'gestion.region'
                  , 'gestion.matricula'
                  , 'gestion.DDA'
                  , 'gestion.grupoDesc'
                  , 'gestion.estadoInstalacion'
                  , '@timestamp']

concat['gestion.tipo'] = "incidente"

concat['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Incidentes-' + str(document['gestion.site_id']) + '-' + str(document['gestion.numeroTicket']) + '-' + str(document['gestion.fechaApertura'])}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(concat))
print("Fecha: ", now,"- Valoraciones insertadas en indice principal:",salida[0])

Actualizando fecha de control de ejecución

In [ ]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, "%Y-%m-%d"'T'"%H:%M:%S")+timedelta(minutes=10)).strftime("%Y-%m-%d"'T'"%H:%M:%S")[0:15] + '0:00'    
#fecha_ejecucion = "2021-05-01T00:00:00"

if fecha_ejecucion > str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime("%Y-%m-%d"'T'"%H:%M:%S"))[0:15] + '0:00'
response = es.index(
        index = parametros.mintic_control,
        id = 'jerarquia_gestion_incidentes',
        body = { 'jerarquia_gestion_incidentes': 'gestion_incindentes','gestion.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)